In [1]:
import numpy as np
import h5py
import sys
import xarray as xr
import os
sys.path.append('../')
import tidy3d as td
import tidy3d_core as tdcore

In [2]:
SIM = td.Simulation(
    size=(2.0, 2.0, 2.0),
    grid_size=.1,
    monitors={
        "field_freq": td.FieldMonitor(size=(1,1,1), center=(0,1,0), sampler=td.FreqSampler(freqs=[1,2, 5, 7, 8])),
        "field_time": td.FieldMonitor(size=(1,1,0), center=(1,0,0), sampler=td.TimeSampler(times=[1])),
        "flux_freq": td.FluxMonitor(size=(1,1,0), center=(0,0,0), sampler=td.FreqSampler(freqs=[1,2, 5, 9])),
        "flux_time": td.FluxMonitor(size=(1,1,0), center=(0,0,0), sampler=td.TimeSampler(times=[1,2, 3])),
        "mode": td.ModeMonitor(size=(1,1,0), center=(0,0,0), sampler=td.FreqSampler(freqs=[1.90,2.01, 2.2]), modes=[td.Mode(mode_index=1)])
    },
)

TMP_DIR = 'notebooks/tmp/'

def _clear_dir(path=TMP_DIR):
    for f in os.listdir(path):
        os.remove(os.path.join(path, f))    

# decorator that clears the tmp/ diretory before test
def clear_tmp(fn):
    def new_fn(*args, **kwargs):
        _clear_dir(TMP_DIR)
        return fn(*args, **kwargs)
    return new_fn
 
def prepend_tmp(path):
    return os.path.join(TMP_DIR, path)


In [3]:
json_path = prepend_tmp('simulation.json')
SIM.export(json_path)
sim_core = tdcore.load_simulation_json(json_path)
solver_data = tdcore.solve(sim_core)    
# sim_data = tdcore.load_solver_results(sim_core, solver_data) 

json_string = sim_core.json()

In [4]:
# write to file
fname = prepend_tmp('data.hdf5')
os.remove(fname)
with h5py.File(fname, 'a') as f:
    mon_data_grp = f.create_group('monitor_data')
    f.attrs['json_string'] = json_string
    for mon_name, mon_data in solver_data.items():
        mon_grp = mon_data_grp.create_group(mon_name)
        for data_name, data_value in mon_data.items():
            mon_grp.create_dataset(data_name, data=data_value)

In [5]:
data_dim_map = {
    td.FieldData: ['field', 'component', 'xs', 'ys', 'zs', 'sampler_value(replace)'],
    td.FluxData: ['sampler_value(replace)'],
    td.ModeData: ['direction', 'mode_index', 'sampler_value(replace)'],
}

In [6]:
# read from file
with h5py.File(fname, 'r') as f:

    # construct the original simulation from the json string
    json_string = f.attrs['json_string']
    sim = td.Simulation.parse_raw(json_string)

    # loop through monitor dataset and create all MonitorData instances
    monitor_data_dict = {}
    monitor_data = f['monitor_data']
    for mon_name, mon_data in monitor_data.items():

        # get info about the original monitor
        monitor = sim.monitors.get(mon_name)
        assert monitor is not None, "monitor not found in original simulation"
        mon_type = type(monitor)
        data_type = td.monitor_data_map[mon_type]
        
        # get the dimensions for this data type, replace sampler data with correct value
        dims = data_dim_map[data_type]
        sampler_dim = 'freqs' if isinstance(monitor.sampler, td.FreqSampler) else 'times'
        dims[-1] = sampler_dim
        
        # load data from dataset, separate data and coordinates 
        coords = {}
        for data_name, dataset in mon_data.items():
            data_value = np.array(dataset)
            coords[data_name] = data_value
        data_value = coords.pop('data')
        
        # load into an xarray.DataArray and make a monitor data to append to dictionary
        darray = xr.DataArray(data_value, coords, dims=dims, name='mon_name')
        monitor_data_instance = data_type(darray)
        for c in monitor_data_instance.coords:
            print(monitor_data_instance.data)
        monitor_data_dict['mon_name'] = monitor_data_instance

    # return a SimulationData object containing all of the information
    s = td.SimulationData(simulation=sim, monitor_data=monitor_data_dict)

[[[[[[-4.31331122e-01-5.07163769e-01j  1.06895165e-01-6.57141575e-01j
       2.48348950e-01+6.17725202e-01j -6.49585281e-01+1.45947210e-01j
      -5.32016639e-01-4.00274807e-01j]
     [-5.20238557e-01-5.49124451e-01j  4.08359284e-02-7.55326582e-01j
       4.58003584e-01+6.02012072e-01j -6.25859551e-01+4.24836016e-01j
      -7.38844798e-01-1.62154810e-01j]
     [-6.08478768e-01-5.87800707e-01j -2.92388271e-02-8.45518286e-01j
       6.47630076e-01+5.44363267e-01j -5.21655787e-01+6.66056545e-01j
      -8.37949325e-01+1.16606222e-01j]
     ...
     [-6.08478768e-01-5.87800707e-01j -2.92388271e-02-8.45518286e-01j
       6.47630076e-01+5.44363267e-01j -5.21655787e-01+6.66056545e-01j
      -8.37949325e-01+1.16606222e-01j]
     [-5.20238557e-01-5.49124451e-01j  4.08359284e-02-7.55326582e-01j
       4.58003584e-01+6.02012072e-01j -6.25859551e-01+4.24836016e-01j
      -7.38844798e-01-1.62154810e-01j]
     [-4.31331122e-01-5.07163769e-01j  1.06895165e-01-6.57141575e-01j
       2.48348950e-01+6.17

[[[[[[-4.31331122e-01-5.07163769e-01j  1.06895165e-01-6.57141575e-01j
       2.48348950e-01+6.17725202e-01j -6.49585281e-01+1.45947210e-01j
      -5.32016639e-01-4.00274807e-01j]
     [-5.20238557e-01-5.49124451e-01j  4.08359284e-02-7.55326582e-01j
       4.58003584e-01+6.02012072e-01j -6.25859551e-01+4.24836016e-01j
      -7.38844798e-01-1.62154810e-01j]
     [-6.08478768e-01-5.87800707e-01j -2.92388271e-02-8.45518286e-01j
       6.47630076e-01+5.44363267e-01j -5.21655787e-01+6.66056545e-01j
      -8.37949325e-01+1.16606222e-01j]
     ...
     [-6.08478768e-01-5.87800707e-01j -2.92388271e-02-8.45518286e-01j
       6.47630076e-01+5.44363267e-01j -5.21655787e-01+6.66056545e-01j
      -8.37949325e-01+1.16606222e-01j]
     [-5.20238557e-01-5.49124451e-01j  4.08359284e-02-7.55326582e-01j
       4.58003584e-01+6.02012072e-01j -6.25859551e-01+4.24836016e-01j
      -7.38844798e-01-1.62154810e-01j]
     [-4.31331122e-01-5.07163769e-01j  1.06895165e-01-6.57141575e-01j
       2.48348950e-01+6.17

[[[[[[-4.31331122e-01-5.07163769e-01j  1.06895165e-01-6.57141575e-01j
       2.48348950e-01+6.17725202e-01j -6.49585281e-01+1.45947210e-01j
      -5.32016639e-01-4.00274807e-01j]
     [-5.20238557e-01-5.49124451e-01j  4.08359284e-02-7.55326582e-01j
       4.58003584e-01+6.02012072e-01j -6.25859551e-01+4.24836016e-01j
      -7.38844798e-01-1.62154810e-01j]
     [-6.08478768e-01-5.87800707e-01j -2.92388271e-02-8.45518286e-01j
       6.47630076e-01+5.44363267e-01j -5.21655787e-01+6.66056545e-01j
      -8.37949325e-01+1.16606222e-01j]
     ...
     [-6.08478768e-01-5.87800707e-01j -2.92388271e-02-8.45518286e-01j
       6.47630076e-01+5.44363267e-01j -5.21655787e-01+6.66056545e-01j
      -8.37949325e-01+1.16606222e-01j]
     [-5.20238557e-01-5.49124451e-01j  4.08359284e-02-7.55326582e-01j
       4.58003584e-01+6.02012072e-01j -6.25859551e-01+4.24836016e-01j
      -7.38844798e-01-1.62154810e-01j]
     [-4.31331122e-01-5.07163769e-01j  1.06895165e-01-6.57141575e-01j
       2.48348950e-01+6.17

[[[[[[-0.75872714-0.64834026j -0.15563243-0.98579436j
       0.52208971-0.85054941j]]

    [[-0.78044685-0.58145597j -0.27845892-0.93254971j
       0.33384935-0.91418431j]]

    [[-0.73439508-0.4844079j  -0.34632542-0.80873102j
       0.1561962 -0.8657886j ]]

    ...

    [[ 0.73439508+0.4844079j   0.34632542+0.80873102j
      -0.1561962 +0.8657886j ]]

    [[ 0.78044685+0.58145597j  0.27845892+0.93254971j
      -0.33384935+0.91418431j]]

    [[ 0.75872714+0.64834026j  0.15563243+0.98579436j
      -0.52208971+0.85054941j]]]


   [[[-0.97555857-0.72681996j -0.34807365-1.16568714j
       0.41731168-1.14273038j]]

    [[-1.0519893 -0.66790603j -0.53011787-1.12772115j
       0.15691558-1.23618666j]]

    [[-1.04890346-0.57301857j -0.64577965-1.00574221j
      -0.08454646-1.19222508j]]

    ...

    [[ 1.04890346+0.57301857j  0.64577965+1.00574221j
       0.08454646+1.19222508j]]

    [[ 1.0519893 +0.66790603j  0.53011787+1.12772115j
      -0.15691558+1.23618666j]]

    [[ 0.97555857+0.726

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
